In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import contextily as cx
import datetime as dt
import json
from joblib import Parallel, delayed
import joblib
import haversine as hs
pd.options.mode.chained_assignment = None  # default='warn'
from matplotlib.collections import LineCollection
import matplotlib.colors as mcol

from functions_file import *
print(testfunction(4))

16


### Outline
1. Prep
2. Calculate Mobile Sensor values
3. Make a loop detector and calculate values

### 1. Prep

In [2]:
# car penetration rates as modes: 5 %
carmodes = ['Car0050'] 
# Assumed vehicle lengths:
lengths = {'Car':5,'Bus':12.5,'Taxi':5,'Motorcycle':2.5,'Medium Vehicle':5.83,'Heavy Vehicle':12.5}
DOW_exp_dict = [['Wed',[1,2,3,4,5]],['Mon',[6,7,9,10]],['Tue',[11,12,13,14,15]],['Thu',[17,18,19,20]]]
polygons = pd.read_csv('../data/polygons11.csv')
# Polygon 4 (road segment 4) is dropped, as it is a different kind of road segment; it is multiple blocks long. Polygons 0-3 and 5-11 are used (11 segments).
polygons = polygons.drop([4])

### 2. Process Mobile Sensor (MS) Data and calculate v, q, k

The car modes ('cars') are calculated separately from all other considered modes ('regular') to account for the various penetration rates.

In [ ]:
######################
# CALCULATE VALUES - based on timestamp, for all polygons
######################
for version in ['regular','cars']: 
    save = 'off' # 'on' for saving the calculations
    for polygon_name in polygons.name:
        
        # load data
        waypoints_w_dist_mode,POLYGON = load_data(polygon_name,polygons)
        # set time interval
        step = dt.timedelta(seconds=30)
        # select version
        save_data = []
        if 'cars'==version:
            modes = carmodes
            savefilename = '../output/data_flow/flow_data_MS_%s_cars.json'%(POLYGON['name'])
        if 'regular'==version:
            modes = np.append( waypoints_w_dist_mode['type'].unique(), ['all'])
            savefilename = '../output/data_flow/flow_data_MS_%s.json'%(POLYGON['name'])

        # 0. per mode
        for mode in modes:

            # 1. select data (the functions are in functions_file.py)
            mode_section,_ = get_mode_section(mode,carmodes,waypoints_w_dist_mode,lengths)
            if not len(mode_section)>0:
                print('No data for ',mode)
                continue
            # 2. process each experiment
            for DOW,day in DOW_exp_dict:

                # regular version
                '''for exp in day:
                    exp_results = process_exp_MS_Edie([mode_section,mode,POLYGON,day,DOW,exp,step])
                    if (exp_results is not None):
                        save_data.append(exp_results)'''
                # parallel version (no output)
                t1 = dt.datetime.now()
                print('Polygon: ',polygon_name,', length: ',POLYGON['length'])
                day_results = Parallel(n_jobs=-2)(delayed(process_exp_MS_Edie)([mode_section,mode,POLYGON,day,DOW,exp,step]) for exp in day) 
                # save
                print('Duration:',dt.datetime.now()-t1,'--> ',DOW,'done')
                for exp_results in day_results:
                    if (exp_results is not None):
                        save_data.append(exp_results)

        # 4. save data
        if save=='on':
            with open(savefilename, 'w', encoding='utf-8') as f: 
                json.dump(save_data, f, default=str)
        save_data = pd.DataFrame(save_data,columns=['polygon','mode','day','DOW','exp_id','speeds','densities','flows','times'])
        display(save_data.head(2))

### 3. Generate LD Data and calculate v, q, k

In [ ]:
######################
# SINGLE LOOP DETECTOR - only q and occ
######################

save = 'off' # 'on' for saving the calculations
for polygon_name in polygons.name: 
    
    waypoints_w_dist_mode,POLYGON = load_data(polygon_name,polygons,file_type='csv')

    # set loop detector location
    location = POLYGON['length']*1000-35 # LD approx. 30-40m from end of segment (signals)
    # set time parameters
    window = None      # --> hour: H, min: T, sec: s
    resample = '30s'   # --> hour: H, min: T, sec: s
    resampleInSec = 30 
    timetohour = 3600/resampleInSec # resample*timetohour = 1hr
    
    # prep
    cs = list(POLYGON['coords'].exterior.coords) # corners
    save_data = []
    modes = ['all'] # DON'T NEED ANY MODES EXCEPT FOR 'ALL'
    savefilename = '../output/data_flow/flow_data_singleLD_%s_%s.json'%(POLYGON['name'],resample)
    
    # 0. per mode
    for mode in modes: 
        
        # 1. select data
        mode_section,vehlength = get_mode_section(mode,carmodes,waypoints_w_dist_mode,lengths)
        if not len(mode_section)>0:
            print('No data for ',mode)
            continue
        
        # 2. process each experiment
        for DOW,day in DOW_exp_dict:
            # regular version
            '''for exp in day:
                exp_results = process_exp_LD([mode_section,mode,POLYGON,day,DOW,exp,location,cs,vehlength,resample,resampleInSec,timetohour,window])
                #if plot=='on':
                #    plt.scatter(LD_info_.density,LD_info_.flow)
                if (exp_results is not None):
                    save_data.append(exp_results)'''
                    
            # parallel version
            exp_results = Parallel(n_jobs=-2)(delayed(process_exp_LD)([mode_section,mode,POLYGON,day,DOW,exp,location,cs,vehlength,resample,resampleInSec,timetohour,window]) for exp in day) # -2 --> all but one proc
            for exp in exp_results:
                if (exp is not None):
                    save_data.append(exp)

    
    # 3. save data
    if save=='on':
        with open(savefilename, 'w', encoding='utf-8') as f:
                json.dump(save_data, f, default=str)
    save_data = pd.DataFrame(save_data,columns=['polygon','mode','day','DOW','exp_id','speeds','densities','flows','times'])
    display(save_data.head(2))